# Introducción

En este notebook se crearán los videos correspondientes a las acciones 'No offence' con el objetivo de compensar el desequilibrio del conjunto de datos de entrenamiento.


In [ ]:
import json
import pandas as pd
from collections import Counter
import os
from collections import defaultdict
import os
import cv2
import numpy as np
from sklearn.utils import resample
import shutil
import vidaug.augmentors as va
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from PIL import ImageOps, Image


In [ ]:
#Funcion de lectura de las etiquetas a partir de un json
def load_foul_labels(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    action_labels = {}
    for action_id, action_data in data['Actions'].items():
        label = action_data['Offence']  # Extraer la etiqueta de 'Offence'
        action_name = f"action_{action_id}"  # Formar el nombre de la acción
        action_labels[action_name] = label
    
    return action_labels

In [ ]:
#Carga de etiquetas en un listado ordenado 
ls = load_foul_labels("F:/data/mvfouls/train/annotations.json")
ls = sorted(ls.items())
sorted_labels_train = sorted(ls, key=lambda x: int(x[0].split('_')[1]))


In [ ]:
sorted_labels_train[:10]

In [ ]:
#Creamos la distribución de clases

df = pd.DataFrame(sorted_labels_train, columns=['action', 'label'])

# Contar la frecuencia de cada clase
class_counts = Counter(df['label'])

print(class_counts)

In [ ]:
#Funcion para agrupar la ruta de los videos por accion  

def group_videos_by_action(root_path, video_extensions=None):
    """
    Agrupa videos por acción según el nombre de las carpetas contenedoras.

    Args:
        root_path (str): Ruta raíz desde donde empezar a buscar.
        video_extensions (list, optional): Lista de extensiones de video a buscar.
                                           Si no se proporciona, usará valores predeterminados.
    Returns:
        dict: Diccionario donde las claves son nombres de acciones (carpetas)
              y los valores son listas de rutas de videos.
    """
    if video_extensions is None:
        video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv']

    grouped_videos = defaultdict(list)

    for dirpath, dirnames, filenames in os.walk(root_path):
        for file in filenames:
            if any(file.lower().endswith(ext) for ext in video_extensions):
                action_name = os.path.basename(dirpath)  
                file_path = os.path.join(dirpath, file)
                grouped_videos[action_name].append(file_path)
    
    sorted_actions = sorted(
        grouped_videos.items(), 
        key=lambda x: int(x[0].split('_')[1])  # Extraer número después de "action_"
    )
    return grouped_videos

root_path = "F:/data/mvfouls/train"  #Ruta de los videos para agrupar
videos_path = group_videos_by_action(root_path = root_path)


In [ ]:
#Funciones para cargar los videos y guardar los nuevos creados a partir de la función de data augmentation.

def load_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return np.array(frames)

def save_video(frames, output_path, fps=30):
    height, width, layers = frames[0].shape
    size = (width, height)
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
    for frame in frames:
        out.write(frame)
    out.release()

#Función que identifica las acciones con etiqueta No offence y crea una lista aleatoria de acciones para editar   
def identify_and_resample_no_offence(actions, labels, target_count):
    no_offence_actions = [action for action, label in labels.items() if label == 'No offence']
    resampled_actions = resample(no_offence_actions, replace=True, n_samples=target_count, random_state=123)
    return resampled_actions




In [ ]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import vidaug.augmentors as va
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from PIL import ImageOps, Image

# Data Augmentation para Videos Etiquetados como 'No offence'

Este código implementa un conjunto de funciones para realizar **data augmentation** en videos etiquetados como `'No offence'`, con el objetivo de equilibrar un conjunto de datos de entrenamiento. A continuación, se describen las transformaciones aplicadas:

## Transformaciones

1. **Transformaciones básicas con `vidaug`**:
   - **Rotación horizontal**: Voltea los frames horizontalmente.
   - **Rotación vertical**: Voltea los frames verticalmente.
   - **Aumento de brillo**: Incrementa el brillo de los frames.

2. **Transformaciones adicionales**:
   - **`add_colored_dots`**: Agrega puntos de colores (rojos, negros y blancos) semi-transparentes a los frames para aumentar la variedad visual.
   - **`smooth_rotation`**: Aplica una rotación fluida a lo largo de los frames, controlando el ángulo máximo y la velocidad de rotación.

## Aplicación en `augment_video`

La función `augment_video` toma una lista de frames y aplica secuencialmente las transformaciones mencionadas, combinando las operaciones de `vidaug` y las funciones adicionales para generar videos con mayor diversidad.

In [ ]:





def adjust_saturation(image, factor=1.5):
    """
    Ajusta la saturación de una imagen usando OpenCV.
    :param image: Imagen en formato numpy array (BGR).
    :param factor: Factor de aumento de saturación.
    :return: Imagen con saturación ajustada.
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
    hsv[..., 1] = np.clip(hsv[..., 1] * factor, 0, 255)  # Aumenta saturación
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)

def adjust_brightness(image, factor=1.2):
    """
    Ajusta el brillo de una imagen multiplicando el canal V en el espacio HSV.
    :param image: Imagen en formato numpy array (BGR).
    :param factor: Factor de aumento o disminución del brillo.
    :return: Imagen con brillo ajustado.
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
    hsv[..., 2] = np.clip(hsv[..., 2] * factor, 0, 255)  # Ajusta el brillo
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)

def add_random_noise(image, intensity=15):
    """
    Agrega ruido gaussiano aleatorio a la imagen.
    :param image: Imagen en formato numpy array.
    :param intensity: Intensidad del ruido.
    :return: Imagen con ruido añadido.
    """
    noise = np.random.randint(-intensity, intensity, image.shape, dtype=np.int16)
    noisy_image = np.clip(image.astype(np.int16) + noise, 0, 255)  # Mantiene valores en rango
    return noisy_image.astype(np.uint8)

def apply_gaussian_blur(image, ksize=(5,5)):
    """
    Aplica un desenfoque gaussiano a la imagen.
    :param image: Imagen en formato numpy array.
    :param ksize: Tamaño del kernel para el desenfoque.
    :return: Imagen suavizada.
    """
    return cv2.GaussianBlur(image, ksize, 0)

def smooth_rotation(frames, max_angle=10, speed_factor=4):
    """
    Aplica una rotación más rápida y fluida a lo largo de los frames.
    :param frames: Lista de frames (numpy arrays).
    :param max_angle: Ángulo máximo de rotación en grados.
    :param speed_factor: Factor de velocidad para hacer la rotación más rápida.
    :return: Lista de frames rotados suavemente.
    """
    num_frames = len(frames)
    angles = max_angle * np.sin(np.linspace(-np.pi * speed_factor, np.pi * speed_factor, num_frames))

    rotated_frames = []
    for i, frame in enumerate(frames):
        angle = angles[i]  # Toma el ángulo correspondiente al frame
        (h, w) = frame.shape[:2]
        M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
        rotated_frame = cv2.warpAffine(frame, M, (w, h), borderMode=cv2.BORDER_REFLECT)  # Evita bordes negros
        rotated_frames.append(rotated_frame)

    return rotated_frames

def add_colored_dots(image, num_dots=1000, dot_size=2, alpha=0.7):
    """
    Agrega puntos rojos, negros y blancos semi-transparentes a la imagen.
    :param image: Frame en formato numpy array (BGR).
    :param num_dots: Número total de puntos a dibujar.
    :param dot_size: Tamaño de cada punto en píxeles.
    :param alpha: Opacidad del efecto (0 = invisible, 1 = sólido).
    :return: Imagen con efecto de puntitos de diferentes colores.
    """
    overlay = image.copy()
    h, w = image.shape[:2]
    colors = [(0, 0, 255), (0, 0, 0), (255, 255, 255)]  # Rojo, Negro, Blanco

    for _ in range(num_dots):
        x, y = random.randint(0, w - 1), random.randint(0, h - 1)
        color = random.choice(colors)  # Selecciona un color aleatorio
        cv2.circle(overlay, (x, y), dot_size, color, -1)

    return cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)  # Mezcla las capas

def augment_video(frames, rotate_every=10):
    """
    Aplica transformaciones de `vidaug`, rota cada cierto número de frames y añade puntos de colores.
    :param frames: Lista de frames (numpy arrays en formato BGR).
    :param rotate_every: Número de frames después del cual se aplica una rotación.
    :return: Lista de frames transformados.
    """
     
    transformed_frames = []

    for i, frame in enumerate(frames):
        if i % rotate_every == 0:  # Rotar cada 'rotate_every' frames
            angle = random.uniform(-5, 5)
            (h, w) = frame.shape[:2]
            M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
            frame = cv2.warpAffine(frame, M, (w, h), borderMode=cv2.BORDER_REFLECT)

        frame = add_colored_dots(frame, num_dots=500, dot_size=2, alpha=0.6)  # Agregar puntitos de colores
        transformed_frames.append(frame)

    return np.array(transformed_frames)

def navigate_and_resample_videos(base_path, actions, resampled_actions):
    for action in resampled_actions:  # Iteramos sobre la lista de acciones resampleadas
        if action in actions:
            video_list = actions[action]  # Obtenemos los videos asociados a la acción
            
            # Crear una única carpeta para guardar todos los videos aumentados de esta acción
            new_action_path = os.path.join(base_path, f"{action}_augmented")
            os.makedirs(new_action_path, exist_ok=True)
            
            for video_file in video_list:
                video_path = os.path.join(base_path, video_file)
                
                frames = load_video(video_path)
                if len(frames) == 0:
                    print(f"Advertencia: No se encontraron frames en {video_path}, saltando...")
                    continue

                augmented_frames = augment_video(frames) 
                
                # Nombre del archivo de salida dentro de la carpeta de la acción
                video_name = os.path.splitext(os.path.basename(video_file))[0]  # Quita la extensión
                output_video_path = os.path.join(new_action_path, f"{video_name}_augmented.mp4")
                
                print(f"Guardando: {output_video_path}")
                save_video(augmented_frames, output_video_path)
                
base_path = 'F:/data/mvfouls/train'
actions= videos_path
actions_labels_dict = {action: label for action, label in sorted_labels_train}
target_count = 200 #Numero objetivo de acciones "No offence" después de resamplear

resampled_actions = identify_and_resample_no_offence(actions, actions_labels_dict, target_count)
navigate_and_resample_videos(base_path, actions, resampled_actions) 

In [ ]:
import os

def create_action_label_dict(base_path):
    action_label_dict = {}  # Diccionario para almacenar las acciones y sus etiquetas

    # Recorremos las carpetas dentro de 'F:/data/mvfouls/train'
    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        
        # Comprobamos si es una carpeta y si el nombre tiene el formato 'action_<id>_augmented'
        if os.path.isdir(folder_path) and folder_name.endswith('_augmented'):
            action_name = folder_name.split('_augmented')[0]  # Extraemos el nombre de la acción
            action_label_dict[action_name] = 'No offence'  # Añadimos la acción con el label 'No offence'

    return action_label_dict

# Ruta base donde se encuentran las carpetas
base_path = 'F:/data/mvfouls/train'

# Crear el diccionario de acciones con su label
action_label_dict = create_action_label_dict(base_path)

# Mostrar el diccionario generado
print(len(action_label_dict))

In [ ]:
#Función para renombrar las carpetas de videos aumentados siguiendo el orden de acciones
def rename_action_folders_sequential(base_path, start_number):
    counter = start_number
    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        
        # Verificamos si es una carpeta y si tiene el sufijo '_augmented'
        if os.path.isdir(folder_path) and folder_name.endswith('_augmented'):
            new_folder_name = f"action_{counter}"
            new_folder_path = os.path.join(base_path, new_folder_name)
            print(f"Renombrando {folder_path} a {new_folder_path}")
            os.rename(folder_path, new_folder_path)
            counter += 1  
    
    print("Renombrado secuencial completo.")


base_path = 'F:/data/mvfouls/train' #Ruta donde estan los videos aumentados


start_number = 3861 #Acción por la que empezar a renombrar
rename_action_folders_sequential(base_path, start_number)


In [ ]:
#AÑADIMOS NUEVAS ACCIONES AL DICCIONARIO DE LABELS 

# Suponiendo que tienes este diccionario de acciones con las etiquetas actuales en 'labels'
# Ejemplo:
# labels = [('action_0', 'Offence'), ('action_1', 'Offence'), ...]

# Calcular cuántas nuevas acciones necesitamos para llegar a 3345
new_actions_needed = 3944 - len(sorted_labels_train)+1

# Generar nuevas acciones con la etiqueta 'No Offence'
new_actions = [('action_' + str(i), 'No offence') for i in range(len(sorted_labels_train), len(sorted_labels_train) + new_actions_needed)]

# Agregar las nuevas acciones al diccionario
updated_labels = sorted_labels_train + new_actions

# Verificar el total final y mostrar algunas de las nuevas acciones
print(f"Total de acciones después de la ampliación: {len(updated_labels)}")
print(updated_labels[-1:])  # Mostrar las últimas 10 acciones para verificar
